# Doubly Robust Estimation

* 変数間の影響が線形の場合に使えr
* Regression AdjustmentとIPTWを組み合わせた方法
* 反実仮想を回帰分析より計算し，IPTW法を使う

In [1]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression

import random
import numpy as np
import pandas as pd


from numpy.random import *
import matplotlib.pyplot as plt
import scipy.stats
from scipy.special import expit


np.random.seed(3655)

## データの用意

In [2]:
# データ数
num_data = 100000

# 年齢
x_1 = randint(15, 76, num_data)  # 15から75歳の一様乱数

# 性別（0を女性、1を男性とします）
x_2 = randint(0, 2, num_data)  # 0か1の乱数

# ノイズの生成
e_z = randn(num_data)

# シグモイド関数に入れる部分
z_base = x_1 + (1-x_2)*10 - 40 + 5*e_z

# シグモイド関数を計算
z_prob = expit(0.1*z_base)

# テレビCMを見たかどうかの変数（0は見ていない、1は見た）
Z = np.array([])

for i in range(num_data):
    Z_i = np.random.choice(2, size=1, p=[1-z_prob[i], z_prob[i]])[0]
    Z = np.append(Z, Z_i)
    
# ノイズの生成
e_y = randn(num_data)

Y = -x_1 + 30*x_2 + 10*Z + 80 + 10*e_y

In [3]:
df = pd.DataFrame({'年齢': x_1,
                   '性別': x_2,
                   'CMを見た': Z,
                   '購入量': Y,
                   })

df.head() 

,年齢,性別,CMを見た,購入量
0,42,1,1.0,80.742174
1,46,1,1.0,78.582308
2,28,0,1.0,69.532577
3,21,0,0.0,73.698435
4,69,1,1.0,44.185729


## 反実仮想の計算

In [4]:
X = df[['年齢','性別','CMを見た']]

y = df['購入量']

reg2 = LinearRegression().fit(X, y)



X_0 = X.copy()
X_0['CMを見た'] = 0
Y_0 = reg2.predict(X_0)



X_1 = X.copy()
X_1['CMを見た'] = 1
Y_1 = reg2.predict(X_1)

## 傾向スコアの出力

In [6]:
X =df[['年齢','性別']]
Z = df['CMを見た']

reg = LogisticRegression().fit(X, Z)

Z_pre = reg.predict_proba(X)
print(Z_pre[0:5])

[[0.45325197 0.54674803]
 [0.36173493 0.63826507]
 [0.54032628 0.45967372]
 [0.69575937 0.30424063]
 [0.05982856 0.94017144]]


## 因果の効果

In [7]:
ATE_1_i = Y/Z_pre[:, 1]*Z + (1-Z/Z_pre[:, 1])*Y_1
ATE_0_i = Y/Z_pre[:, 0]*(1-Z) + (1-(1-Z)/Z_pre[:, 0])*Y_0
ATE = 1/len(Y)*(ATE_1_i-ATE_0_i).sum()

print("推定したATE", ATE)

推定したATE 9.953203491600581
